In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import re
import emoji
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from sklearn.model_selection import train_test_split

In [2]:
df=pd.read_csv('Tweets.csv')
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [3]:
df.shape 

(14640, 15)

In [4]:
df.drop('tweet_id',axis=1,inplace=True)

In [5]:
df.head()

,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [6]:
for col in df.columns :
    print(col)
    print(df[col].value_counts())
    print('=================================================================================')

airline_sentiment
negative    9178
neutral     3099
positive    2363
Name: airline_sentiment, dtype: int64
airline_sentiment_confidence
1.0000    10445
0.6667       71
0.6632       35
0.6559       30
0.6596       30
0.6701       29
0.6598       28
0.6600       26
0.6842       26
0.6771       24
0.6739       22
0.6737       22
0.6630       22
0.6421       22
0.6809       22
0.6774       22
0.6735       21
0.6522       20
0.6566       19
0.6633       19
0.6703       19
0.6563       18
0.6768       17
0.6742       17
0.6634       17
0.6593       17
0.6889       16
0.6591       16
0.6832       16
0.6804       15
          ...  
0.3537        1
0.6958        1
0.3708        1
0.3533        1
0.6373        1
0.3666        1
0.9286        1
0.6155        1
0.3606        1
0.3661        1
0.3486        1
0.3626        1
0.9220        1
0.3612        1
0.7145        1
0.6102        1
0.6284        1
0.6159        1
0.7134        1
0.6486        1
0.6586        1
0.3493        1
0.6355        1


Name: tweet_created, Length: 14247, dtype: int64
tweet_location
Boston, MA                        157
New York, NY                      156
Washington, DC                    150
New York                          127
USA                               126
Chicago                           104
New York City                      96
Los Angeles, CA                    96
NYC                                95
San Francisco, CA                  91
San Francisco                      86
Chicago, IL                        81
Brooklyn, NY                       66
Austin, TX                         64
Los Angeles                        64
Washington, D.C.                   63
Boston                             62
Dallas, TX                         54
Washington DC                      53
Nashville, TN                      45
NY                                 42
Texas                              42
Philadelphia, PA                   38
San Diego                          38
Denver, CO              

*notes about the data :

-- > out of 14,000 approximalty all of them expressed their sentiment , most are negative

-- > 10,000 where confident about their sentiment 

-- > for each negative sentiment there is a reason and a confidence in that reason 3000 , out of 9000 where 100% confident of their reasoning

-- > airlines share of sentiments is equally divided , but virgin america have the thinest share 

-- > 40 gold_cunstomers -- with their reasons for negativity 

-- > same customers tweeted their sentiment several times 

-- > retweet count of a tweet gives an indication of it's impact ( whether it is main stream opinion , made in time of crisis or holidays)

--- > there are duplicated text , cleaning removing the tags 

-- > the tweet_cord t0 be [0,0] can be indication of error , check tweet location for each , tweet locations need to be cleaned 
overall 

-- > tweets are created within febraury of 2015 ( is that month differ in airflights overall)




In [7]:

for col in df.columns :
    print(col)
    print(df[col].isna().sum())
    print('=============================================================================')

airline_sentiment
0
airline_sentiment_confidence
0
negativereason
5462
negativereason_confidence
4118
airline
0
airline_sentiment_gold
14600
name
0
negativereason_gold
14608
retweet_count
0
text
0
tweet_coord
13621
tweet_created
0
tweet_location
4733
user_timezone
4820


In [8]:
df=df[df['airline_sentiment_gold'].isna()]

In [9]:
df.drop(['airline_sentiment_gold','negativereason_gold'],axis=1,inplace=True)

In [13]:
#df[df['negativereason'].isna()]['negativereason']='Not stated'
df.head()

,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,name,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,neutral,1.0000,NaN,NaN,Virgin America,cairdin,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,positive,0.3486,NaN,0.0000,Virgin America,jnardino,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,neutral,0.6837,NaN,NaN,Virgin America,yvonnalynn,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,negative,1.0000,Bad Flight,0.7033,Virgin America,jnardino,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,negative,1.0000,Can't Tell,1.0000,Virgin America,jnardino,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [14]:
df['negativereason'].fillna('Not stated',inplace=True)

In [15]:
df['negativereason'].isna().sum()

0

In [16]:
df['negativereason_confidence'].fillna(0.0,inplace=True)

In [17]:
df_loc=df[~(df['tweet_coord'].isna() & df['tweet_location'].isna() & df['user_timezone'].isna())]

In [18]:
df.drop(['tweet_coord','tweet_location','user_timezone'],axis=1,inplace=True)

In [19]:
df.head()

,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,name,retweet_count,text,tweet_created
0,neutral,1.0000,Not stated,0.0000,Virgin America,cairdin,0,@VirginAmerica What @dhepburn said.,2015-02-24 11:35:52 -0800
1,positive,0.3486,Not stated,0.0000,Virgin America,jnardino,0,@VirginAmerica plus you've added commercials t...,2015-02-24 11:15:59 -0800
2,neutral,0.6837,Not stated,0.0000,Virgin America,yvonnalynn,0,@VirginAmerica I didn't today... Must mean I n...,2015-02-24 11:15:48 -0800
3,negative,1.0000,Bad Flight,0.7033,Virgin America,jnardino,0,@VirginAmerica it's really aggressive to blast...,2015-02-24 11:15:36 -0800
4,negative,1.0000,Can't Tell,1.0000,Virgin America,jnardino,0,@VirginAmerica and it's a really big bad thing...,2015-02-24 11:14:45 -0800


In [20]:
df.describe()

,airline_sentiment_confidence,negativereason_confidence,retweet_count
count,14600.000000,14600.000000,14600.000000
mean,0.900012,0.458315,0.082671
std,0.162977,0.401096,0.746672
min,0.335000,0.000000,0.000000
25%,0.692300,0.000000,0.000000
50%,1.000000,0.624400,0.000000
75%,1.000000,0.703025,0.000000
max,1.000000,1.000000,44.000000


In [21]:
df['text'].iloc[18]

'I ❤️ flying @VirginAmerica. ☺️👍'

In [22]:


def clean_text(row):
        reg=re.compile(r'@[\w]*')
        
        row=RE_EMOJI.sub(r'',row)
        
def extract_text(strr):
    reg=re.compile(r'(@[\w]*)|(http[s]?:\/\/(www\.)?(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+)')
    emoji_list=[c for c in strr if c in emoji.UNICODE_EMOJI]
    filtred  = [strr for strr in strr.split() if not any(i in strr for i in emoji_list)]
    clean_text=' '.join(filtred)
    clean_text=reg.sub(r'', clean_text)
    return pd.Series([clean_text,emoji_list],index=['text', 'emojis'])


In [23]:
stry=' I love this graphic. http://t.co/UT5GrRwAaA @abcd.'

extract_text(stry)

text      I love this graphic.  .
emojis                         []
dtype: object

In [24]:

df[['text','emojis']]=df['text'].apply(extract_text)


In [25]:
df['emojis']=df['emojis'].apply(lambda x :  x if x else 0)

In [26]:
df['text'].iloc[7]

' Really missed a prime opportunity for Men Without Hats parody, there. '

In [27]:
stry='I flying are bats'

def get_wordnet_pos(word):
    

    tag =nltk.pos_tag([word])[0][1][0]
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    get=tag_dict.get(tag, wordnet.NOUN)
    return get



def lemma(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = (stopwords.words('english'))
    stop_words.append('I')
  
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)

    tokens_list=[ token for token in tokens if token not in stop_words]
    return ([lemmatizer.lemmatize(w, get_wordnet_pos(w))  for w in tokens_list])


In [28]:


df['text'].iloc[7]

' Really missed a prime opportunity for Men Without Hats parody, there. '

In [29]:
df['text']=df['text'].apply(lemma)

In [92]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
# tokenize and build vocab
vectorizer.fit(df['text_str'])
vector = vectorizer.transform(df['text_str'])
print(vector.shape)

(14600, 11672)


In [41]:
list_w=df['text'].iloc[7]

def word2str(words):
    return ' '.join(word for word in words)
word2str(list_w)

df['text_str']=df['text'].apply(word2str)

In [51]:
#y = lambda symbol: 'X' if symbol==True else 'O' if symbol==False else ' '

df['sentiment_encode']=df['airline_sentiment'].apply(lambda x : 2 if x =='positive' else 1 if x =='neutral' else 0)


In [216]:
x_train, x_test, y_train, y_test = train_test_split(df['text_str'], df['sentiment_encode'], test_size=0.2)

In [217]:
x_train.shape , y_train.shape , x_test.shape,y_test.shape

((11680,), (11680,), (2920,), (2920,))

In [121]:

from sklearn.feature_extraction.text import HashingVectorizer
# create the transform
vectorizer = HashingVectorizer(n_features=20000)
# encode document
train_features = vectorizer.transform(x_train)
test_features= vectorizer.transform(x_test)
# summarize encoded vector
print(train_features.shape)
print(test_features.toarray())

print(test_features.shape)
print(train_features.toarray())

(11680, 20000)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(2920, 20000)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [66]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score

In [93]:
Classifiers = [
    LogisticRegression(C=0.000000001,solver='liblinear',max_iter=200),
    KNeighborsClassifier(3),
    SVC(kernel="rbf", C=0.025, probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(n_estimators=200),
    AdaBoostClassifier(),
    GaussianNB()]

In [95]:
dense_features=train_features.toarray()
dense_test= test_features.toarray()
Accuracy=[]
Model=[]
for classifier in Classifiers:
    try:
        fit = classifier.fit(train_features,y_train)
        pred = fit.predict(test_features)
        
    except Exception:
        fit = classifier.fit(dense_features,y_train)
        pred = fit.predict(dense_test)
    accuracy = accuracy_score(pred,y_test)
    Accuracy.append(accuracy)
    Model.append(classifier.__class__.__name__)
    print('Accuracy of '+classifier.__class__.__name__+'is '+str(accuracy))  

E:\Anaconda3\envs\data\lib\site-packages\sklearn\linear_model\logistic.py:459: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy of LogisticRegressionis 0.6376712328767123
Accuracy of KNeighborsClassifieris 0.6965753424657535


E:\Anaconda3\envs\data\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Accuracy of SVCis 0.6376712328767123
Accuracy of DecisionTreeClassifieris 0.711986301369863
Accuracy of RandomForestClassifieris 0.760958904109589
Accuracy of AdaBoostClassifieris 0.7027397260273973
Accuracy of GaussianNBis 0.4469178082191781


In [126]:
#from gensim.models.word2vec import Word2Vec
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
#model =gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
#model.most_similar(positive=['woman', 'king'], negative=['man'], topn=5)

In [112]:
TaggedDocument = gensim.models.doc2vec.TaggedDocument

In [113]:
def labelizeReviews(reviews, label_type):
    labelized = []
    for i,v in enumerate(reviews):
        label = '%s_%s'%(label_type,i)
        labelized.append(TaggedDocument(v, [label]))
    return labelized


In [122]:
x_train = labelizeReviews(x_train, 'TRAIN')
x_test = labelizeReviews(x_test, 'TEST')

In [131]:
data=df['text_str']

In [132]:
data = labelizeReviews(data, 'Data')

In [134]:
data[0:10]

[TaggedDocument(words='What say', tags=['Data_0']),
 TaggedDocument(words='plus add commercial experience tacky', tags=['Data_1']),
 TaggedDocument(words='today Must mean need take another trip', tags=['Data_2']),
 TaggedDocument(words='really aggressive blast obnoxious entertainment guest face amp little recourse', tags=['Data_3']),
 TaggedDocument(words='really big bad thing', tags=['Data_4']),
 TaggedDocument(words='seriously would pay 30 flight seat play really bad thing fly VA', tags=['Data_5']),
 TaggedDocument(words='yes nearly every time fly VX ear worm go away', tags=['Data_6']),
 TaggedDocument(words='Really miss prime opportunity Men Without Hats parody', tags=['Data_7']),
 TaggedDocument(words='Well NOW DO D', tags=['Data_8']),
 TaggedDocument(words='amaze arrive hour early You good', tags=['Data_9'])]

In [135]:
import random

size = 400
#instantiate our DM and DBOW models

model_dbow =Doc2Vec(min_count=1, window=10, vector_size=size, sample=1e-3, negative=5, dm=0, workers=3)

#build vocab over all reviews

model_dbow.build_vocab(data)



In [149]:
df['text'].shape[0]

14600

In [ ]:
all_train_reviews

In [ ]:
#We pass through the data set multiple times, shuffling the training reviews each time to improve accuracy.

all_train_reviews =np.asarray(x_train, dtype=object)

for epoch in range(3):
    
    perm = np.random.permutation(np.arange(11680))
    model_dbow.train(all_train_reviews[perm],total_examples=11680,epochs=1)
    print('model_dbow.train', model_dbow.train)

In [ ]:

#Get training set vectors from our models
def getVecs(model, corpus, size):
    vecs = [np.array(model[z.labels[0]]).reshape((1, size)) for z in corpus]
    return np.concatenate(vecs)

train_vecs_dbow = getVecs(model_dbow, x_train, size)

#train over test set

x_test = np.array(x_test)
for epoch in range(10):
    perm = np.random.permutation(x_test.shape[0])
    model_dbow.train(x_test[perm])
#Construct vectors for test reviews

test_vecs_dbow = getVecs(model_dbow, x_test, size)
